### 1 - Definição do problema

TODO

### 2 - Aquisição dos dados
Os dados utilizados para este trabalho foram retirados do Kaggle em https://www.kaggle.com/fedesoriano/heart-failure-prediction

### 3 - Data Wrangling
Importar as bibliotecas que vamos utilizar

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Machine Learning algorithms

In [6]:
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.model_selection import train_test_split, GridSearchCV #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder


In [9]:
df = pd.read_csv('../data/heart.csv', sep=",", encoding="utf-8")
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


The dataset doesnt contain any nulls.
Now lets take a look at the non-metric columns

In [5]:
categorical = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]

for value in categorical:
    print(df[value].value_counts(normalize=True), "\n")

M    0.78976
F    0.21024
Name: Sex, dtype: float64 

ASY    0.540305
NAP    0.221133
ATA    0.188453
TA     0.050109
Name: ChestPainType, dtype: float64 

Normal    0.601307
LVH       0.204793
ST        0.193900
Name: RestingECG, dtype: float64 

N    0.595861
Y    0.404139
Name: ExerciseAngina, dtype: float64 

Flat    0.501089
Up      0.430283
Down    0.068627
Name: ST_Slope, dtype: float64 



Since the column values seem to be specific categories, they can be converted in numerical values to facilitate processing

In [10]:
enc = LabelEncoder()

# Gender: M -> 1 | F -> 0
enc.fit(df['Sex'])
df['Sex'] = enc.transform(df['Sex'])

# ChestPainType: ASY -> 0 | ATA -> 1 | NAP -> 2 | TA -> 3
enc.fit(df['ChestPainType'])
df['ChestPainType'] = enc.transform(df['ChestPainType'])

# RestingECG: LVH -> 0 | Normal -> 1 | ST -> 2
enc.fit(df['RestingECG'])
df['RestingECG'] = enc.transform(df['RestingECG'])

# ExerciseAngina: N -> 0 | Y -> 1
enc.fit(df['ExerciseAngina'])
df['ExerciseAngina'] = enc.transform(df['ExerciseAngina'])

# ST_Slope: Down -> 0 | Flat -> 1 | Up -> 2
enc.fit(df['ST_Slope'])
df['ST_Slope'] = enc.transform(df['ST_Slope'])

df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,110,264,0,1,132,0,1.2,1,1
914,68,1,0,144,193,1,1,141,0,3.4,1,1
915,57,1,0,130,131,0,1,115,1,1.2,1,1
916,57,0,1,130,236,0,0,174,0,0.0,1,1
